<a href="https://colab.research.google.com/github/jiadmiftx/My-Essay-Plants-Identifier/blob/master/Skripsi_Labs_fix_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import main library
import matplotlib.pyplot as plt
from PIL import Image

#Import pytorch library
from torch.nn import Module
from torch.utils.data import Dataset, DataLoader
from torch.functional import F
from pathlib import Path
import torchvision.transforms as trfm
from torch import nn
import torch.optim as optim
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "drive/My Drive/skripsi/"

/content/drive/My Drive/skripsi


In [ ]:
ls

02-entire_model.pt  dataloss.csv                mobilenet1.csv
assets/             datalossD1.csv              mobilenet-fix.pt
coba.pth            dataset-daun10/             mobilenets.csv
dataloss0001.csv    entire_model.pt             mobilenettest.csv
dataloss001.csv     fixdahmodel.pt              modelfixfix.pt
dataloss1.csv       fix_model.pt                state_dictentire_model.pt
dataloss2.csv       fixmodel.pt                 state_dictfix_model.pt
datalossA1.csv      latest.pth                  state_dict_mobile_model.pt
datalossB1.csv      mobile_CPU_mobile_model.pt  state_dict_model.pt
datalossB2.csv      mobile_model_jit.pt
datalossC1.csv      mobile_model.pt


In [ ]:
p = Path('dataset-daun10/train')

In [ ]:
# Melihat semua isi dari folder p
list(p.glob('*'))

[PosixPath('dataset-daun10/train/Tin_Ara'),
 PosixPath('dataset-daun10/train/Gingseng_Jawa'),
 PosixPath('dataset-daun10/train/Kumis_Kucing'),
 PosixPath('dataset-daun10/train/Bugenvil_Putih'),
 PosixPath('dataset-daun10/train/Keji_Beling'),
 PosixPath('dataset-daun10/train/Daun_Sirih'),
 PosixPath('dataset-daun10/train/Sembung'),
 PosixPath('dataset-daun10/train/Buah_Ajaib'),
 PosixPath('dataset-daun10/train/Kumquat_Nagami'),
 PosixPath('dataset-daun10/train/Pukul_Empat')]

In [ ]:
list(enumerate(list(p.iterdir())))

[(0, PosixPath('dataset-daun10/train/Tin_Ara')),
 (1, PosixPath('dataset-daun10/train/Gingseng_Jawa')),
 (2, PosixPath('dataset-daun10/train/Kumis_Kucing')),
 (3, PosixPath('dataset-daun10/train/Bugenvil_Putih')),
 (4, PosixPath('dataset-daun10/train/Keji_Beling')),
 (5, PosixPath('dataset-daun10/train/Daun_Sirih')),
 (6, PosixPath('dataset-daun10/train/Sembung')),
 (7, PosixPath('dataset-daun10/train/Buah_Ajaib')),
 (8, PosixPath('dataset-daun10/train/Kumquat_Nagami')),
 (9, PosixPath('dataset-daun10/train/Pukul_Empat'))]

In [ ]:
#One hot encoding, membuat kelas masing2 data
one_hot = {x[1].name:x[0] for x in list(enumerate(list(p.iterdir())))}
one_hot

{'Buah_Ajaib': 7,
 'Bugenvil_Putih': 3,
 'Daun_Sirih': 5,
 'Gingseng_Jawa': 1,
 'Keji_Beling': 4,
 'Kumis_Kucing': 2,
 'Kumquat_Nagami': 8,
 'Pukul_Empat': 9,
 'Sembung': 6,
 'Tin_Ara': 0}

In [ ]:
# Membuat dataset
class DatasetDaun(Dataset):
  {x[1].name:x[0] for x in list(enumerate(list(p.iterdir())))}
  def __init__(self, path, trfm=None):
    self.path = Path(path)
    self.images = list(self.path.glob("*/*.jpg"))
    self.trfm = trfm
    self.do_one_hot_encoding()
  
  def do_one_hot_encoding(self):
    self.one_hot = {x[1].name:x[0] for x in list(enumerate(list(p.iterdir())))}

  def __len__(self):
    return len(self.images)
  
  def __getitem__(self, i):
    path_image = self.images[i]
    image = Image.open(path_image)
    target = self.one_hot[path_image.parent.name]
    if self.trfm != None:
      image = self.trfm(image)
    return image, target
  

In [ ]:
train_trfm = trfm.Compose([
                           trfm.Resize((300, 300)),
                           trfm.CenterCrop((224, 224)),
                           trfm.RandomHorizontalFlip(),
                           trfm.RandomVerticalFlip(),
                           trfm.RandomRotation(30),
                           trfm.CenterCrop((224, 224)),
                           trfm.RandomHorizontalFlip(),
                           trfm.RandomVerticalFlip(),
                           trfm.RandomRotation(30),
                           trfm.ToTensor(),
                           trfm.Normalize(mean=(0.3843, 0.5313, 0.2795), std=(0.1838, 0.1798, 0.1811))
])

valid_trfm = trfm.Compose([
                           trfm.Resize((300, 300)),
                           trfm.CenterCrop((224, 224)),
                           trfm.ToTensor()
])

In [ ]:
train_set = DatasetDaun("dataset-daun10/train/", train_trfm)

In [ ]:
ls dataset-daun10/

test/  train/


In [ ]:
valid_set = DatasetDaun("dataset-daun10/test/", train_trfm)

In [ ]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)

In [ ]:
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True)

In [ ]:
train_loader_iter = iter(train_loader)

In [ ]:
valid_loader_iter = iter(valid_loader)

In [ ]:
train_loader_iter.next()[0].shape

torch.Size([16, 3, 224, 224])

In [ ]:
# Model buat sendiri, gagal.. jangan dipakai
class ModelDeteksiDaun(nn.Module):
    
    def __init__(self):
        super().__init__()
        # untuk mempelajari feature
        self.cnn = nn.Sequential(
            #inputan channel = 3, output = 16, kernel = 3 -> 3x3
            nn.Conv2d(3, 16, 3),
            nn.MaxPool2d(3),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3),
            nn.MaxPool2d(3),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            # maxpool dimensi 3 -> 3x3
            nn.MaxPool2d(3),
            nn.ReLU()
        )
        # Fully connected layer / classifiernya
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3136, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, 100),
            nn.ReLU(),
            nn.Linear(100, 3)
        )
        
    def forward(self, x):
        cnn = self.cnn(x)
        return self.fc(cnn)x

In [ ]:
mymodel = ModelDeteksiDaun()

In [ ]:
#Sdah dilakukan diatas
train_set = DatasetDaun("dataset-daun/train", train_trfm)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
train_loader_iter = iter(train_loader)

In [ ]:
# Sudah dilakukan diatas
valid_set = DatasetDaun("dataset-daun/test", train_trfm)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True)
valid_loader_iter = iter(valid_loader)

In [ ]:
# Fungsi activasi
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mymodel.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
for epoch in range(25):

    train_loss = []
    valid_loss = []
    train_acc = []
    valid_acc = []

    for index, train in enumerate(train_loader):
        image, target = train

        # Mengahpus gradien dari backward sebelumnya / set gradient menjadi 0 
        optimizer.zero_grad()
        forward = mymodel(image)
        
        # Menghitung Cross Entropy Loss
        loss = criterion(forward, target)

        # What is this? 
        loss.backward()

        # Mengoptimalkan gradient / mengupdate weight pada gradient berdasar gradient setalh backward
        optimizer.step()

        y_pred = forward.argmax(1)

        # Menghitung Accuracy
        acc = sum(1*(target == y_pred))/(len(target)+.0)
        train_loss.append(loss.item())
        train_acc.append(acc)

    for index, valid in enumerate(valid_loader):
        image, target = valid

        forward = mymodel(image)

        loss = criterion(forward, target)

        y_pred = forward.argmax(1)
        acc = sum(1*(target == y_pred))/(len(target)+.0)
        valid_loss.append(loss.item())
        valid_acc.append(acc)
        
    print(f"Epoch {epoch} - Train Loss {sum(train_loss)/len(train_loss)} - Valid loss {sum(valid_loss)/len(valid_loss)} - Train accuracy {(sum(train_acc)/len(train_acc))*100} - Valid accuracy {(sum(valid_acc)/len(valid_acc))*100}")



Epoch 0 - Train Loss 0.3033936669429143 - Valid loss 0.5623985081911087 - Train accuracy 87.5 - Valid accuracy 79.16667175292969
Epoch 1 - Train Loss 0.2257735716799895 - Valid loss 0.4479406327009201 - Train accuracy 94.27082824707031 - Valid accuracy 80.72917175292969
Epoch 2 - Train Loss 0.19631291615466276 - Valid loss 0.46286557987332344 - Train accuracy 93.75 - Valid accuracy 77.08332824707031
Epoch 3 - Train Loss 0.2699238285422325 - Valid loss 0.38567470759153366 - Train accuracy 90.10417175292969 - Valid accuracy 89.0625
Epoch 4 - Train Loss 0.20314741930148253 - Valid loss 0.38901928439736366 - Train accuracy 89.58332824707031 - Valid accuracy 85.9375
Epoch 5 - Train Loss 0.20889762416481972 - Valid loss 0.3072242774069309 - Train accuracy 90.625 - Valid accuracy 86.97917175292969
Epoch 6 - Train Loss 0.1780566486219565 - Valid loss 0.5030258223414421 - Train accuracy 96.35417175292969 - Valid accuracy 83.85417175292969
Epoch 7 - Train Loss 0.16389910504221916 - Valid loss 0.

In [ ]:
loaded = torch.load('latest.pth')

AttributeError: ignored

In [ ]:
ls -alhj

total 47M
drwx------ 4 root root 4.0K Jul 22 06:56 dataset-daun/
-rw------- 1 root root  32M Jul 22 06:33 dataset-daun.zip
-rw------- 1 root root  15M Jul 30 11:21 latest.pth
-rw------- 1 root root  21K Aug 24  2015 Vd-Orig.png


In [ ]:
pred = mymodel(input)
y_pred = pred.argmax(1)


NameError: ignored

In [ ]:
sum(1*(target == y_pred))/(len(target)+.0)

tensor(0.7500)

[0, 1, 1, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1]

In [ ]:
y_pred.numpy()

array([0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])